<a href="https://colab.research.google.com/github/picazio/HUGWork/blob/master/CODBAR_HUG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import all libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
%matplotlib inline

Import dataframe and explore it

In [9]:
url="/Users/euclide/Downloads/REDCap_clean_08_05.xls"
df = pd.read_excel(io=url, sheet_name='Sheet1')

df.head()

,Codbar,id_REDCap,birthday,age,sexe,cancer,diabete,faiblesse_imm,hypertension,maladie_cvd,...,inquiet_fam_3,inquiet_fam_4,inquiet_fam_5,inquiet_fam_7,inquiet_fam_8,inquiet_fam_9,inquiet_fam_6,autre_raison_fam_inquiet_1,consentement,Resp_REDCap
0,110501,2637,1957-06-18,62.0,Masculin,Non,Non,Non,Non,Non,...,Non,Non,Non,Non,Non,Non,Non,NaN,Oui,Rempli en ligne
1,110822,333,1965-10-06,54.0,Féminin,Non,Non,Non,Non,Non,...,Non,Non,Non,Oui,Non,Non,Non,NaN,Oui,Rempli en papier forme
2,110826,923,1943-06-28,76.0,Féminin,Non,Non,Non,Oui,Non,...,Non,Non,Non,Non,Non,Non,Oui,"Ils vivent en Suisse, mais loin de moi",Oui,Rempli en ligne
3,110879,341,1965-09-03,54.0,Féminin,Non,Non,Non,Oui,Non,...,Non,Non,Non,Oui,Non,Non,Non,NaN,Oui,Rempli en papier forme
4,110926,1110,1949-04-21,70.0,Masculin,Non,Non,Non,Oui,Non,...,Oui,Non,Oui,Non,Non,Non,Non,NaN,Oui,Rempli en papier forme


Define some criteria to indentify families. There are old patients and new patients

Define a fuction containing the critetia to create the new column of number of people in each household

In [10]:
def create_households(x) :
    listold=['11','21','31']
    listold2=['410','411']
    if len(str(x))==6 :
        return str(x)
    elif (any(c in str(x)[:2] for c in listold) and len(str(x))==7) :
        return str(x)[:-1]
    elif len(str(x))==8 :
        return str(x)[:-1]
    elif (any(c in str(x)[:3] for c in listold2) and len(str(x))==7) :
        return str(x)[:-1]
    else :
        return str(x)

Apply the function to produce the 'Households_codbar' column

In [11]:
df['Household_codbar']=df['Codbar'].apply(lambda x: create_households(int(x)))

df.head()

,Codbar,id_REDCap,birthday,age,sexe,cancer,diabete,faiblesse_imm,hypertension,maladie_cvd,...,inquiet_fam_4,inquiet_fam_5,inquiet_fam_7,inquiet_fam_8,inquiet_fam_9,inquiet_fam_6,autre_raison_fam_inquiet_1,consentement,Resp_REDCap,Household_codbar
0,110501,2637,1957-06-18,62.0,Masculin,Non,Non,Non,Non,Non,...,Non,Non,Non,Non,Non,Non,NaN,Oui,Rempli en ligne,110501
1,110822,333,1965-10-06,54.0,Féminin,Non,Non,Non,Non,Non,...,Non,Non,Oui,Non,Non,Non,NaN,Oui,Rempli en papier forme,110822
2,110826,923,1943-06-28,76.0,Féminin,Non,Non,Non,Oui,Non,...,Non,Non,Non,Non,Non,Oui,"Ils vivent en Suisse, mais loin de moi",Oui,Rempli en ligne,110826
3,110879,341,1965-09-03,54.0,Féminin,Non,Non,Non,Oui,Non,...,Non,Non,Oui,Non,Non,Non,NaN,Oui,Rempli en papier forme,110879
4,110926,1110,1949-04-21,70.0,Masculin,Non,Non,Non,Oui,Non,...,Non,Oui,Non,Non,Non,Non,NaN,Oui,Rempli en papier forme,110926


Now for each patient 0 we want to count the number of "family members" that have been tested

In [12]:
df['Family_size']=df['Household_codbar'].map(df['Household_codbar'].value_counts())

Let's check if it works

In [13]:
#df[df['Codbar'].apply(lambda x: len(str(x))>=7)]
df[2992:2999]

,Codbar,id_REDCap,birthday,age,sexe,cancer,diabete,faiblesse_imm,hypertension,maladie_cvd,...,inquiet_fam_5,inquiet_fam_7,inquiet_fam_8,inquiet_fam_9,inquiet_fam_6,autre_raison_fam_inquiet_1,consentement,Resp_REDCap,Household_codbar,Family_size


Save to a new excel file

In [14]:
df.to_excel("/Users/euclide/Desktop/REDCap_clean_08_05a.xls")  

Sanity check:

In [15]:
df[df['Family_size']>5]

,Codbar,id_REDCap,birthday,age,sexe,cancer,diabete,faiblesse_imm,hypertension,maladie_cvd,...,inquiet_fam_5,inquiet_fam_7,inquiet_fam_8,inquiet_fam_9,inquiet_fam_6,autre_raison_fam_inquiet_1,consentement,Resp_REDCap,Household_codbar,Family_size
